# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification

## 1. DATA READ


In [20]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")

Dataset.keys()

Index(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       ...
       'n2044', 'n2045', 'n2046', 'n2047', 'category', 'image name', 'image',
       'size', 'width', 'height'],
      dtype='object', length=2054)

In [21]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
DataFrameFlowers = pd.DataFrame(Dataset)


In [23]:
DataFrameFlowersX = DataFrameFlowers.drop(['category'], axis = 1)
X = DataFrameFlowersX
Y = DataFrameFlowers['category']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)


In [24]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)

In [25]:
LR.fit(X_train,y_train)

LogisticRegression(C=0.24875621890547264, penalty='l1', solver='saga', tol=0.1)